## A Particle Zoo 
Build a system of classes that is able to represent particles and their properties!

### 1. Generic particle

Define a `Particle` class to represent a generic particle.

The class should provide access to the following constant properties by means of getter methods:
- rest mass (in eV);
- electric charge;
- spin;
- is stable or not;
- can be observed in isolation or not.

The class should allow to set the following modifiable properties by means of a setter method:
- momentum (scalar, in eV).


### Solution

**General note** : there is not a unique correct way of developing the classes. In particular, how to distribute auxiliary methods between parent and subclasses, as well as how/where to store the particle properties are problems that allow for different solutions.

The definition of the class at this level is relatively plain. We could decide that this class should act only as a parent class and is not meant to be instantiated directly. Such concept is known as "abstract class" and python provides a library to implement it, but this is an advanced topic we can skip for the time being.
Notes:
- momentum could be initialised in the constructor, but to better highlight its nature of mutable attribute (in contrast to the other meant to be immutable), we initialise it by default at zero and se allow setting it by means of a setter method;
- we decide to name getter methods that return boolean values starting with the name `is_` rather than `get_`.

In [ ]:
class Particle:
    def __init__(self, id, mass, charge, spin, stable=None, always_bound=None, antiparticle=False):
        self.id = id
        self.mass = mass
        self.charge = charge
        self.spin = spin
        self.stable = stable
        self.always_bound = always_bound
        self.momentum = 0

    def get_id(self):
        return self.id

    def get_charge(self):
        return self.charge

    def get_spin(self):
        return self.spin

    def is_stable(self):
        return self.stable
    
    def is_always_bound(self):
        return self.always_bound

    def get_momentum(self):
        return self.momentum

    def set_momentum(self, momentum):
        self.momentum = momentum
        
    def __str__(self):
        return f"Particle: {self.get_id()} (m = {self.mass:.1e} eV, chg = {self.get_charge():2f}, spin = {self.get_spin()})"

### 2. Elementary particles
Define a class `ElementaryParticle` derived from `Particle`.

Then, derive from `ElementaryParticle` separate classes to represent their different categories:
- `class Quark` (up, down, top, bottom, charm, strange)
- `class Lepton` (electron, muon, tau, electron neutrino, tau neutrino, muon neutrino)
- `class GaugeBoson` (gluon, photon, Z boson, W boson)
- `class ScalarBoson` (Higgs boson)

Instructions:
- choose an unique identifier (a single letter or short string) for each type of particle in a category;
- use a class to represent a category, as suggested;
- use an attribute to identify a particle in its own category (it can be the same as the unique identifier);
- use an attribute to signal if the particle is an antiparticle; remember that an antiparticle has opposite charge but all the other constant properties are identical!
- provide each class with a class method `from_id(id)` that takes as an argument the unique identifier of a particle and returns an *instance* of such particle! This is probably where you should take care of assigning the properties (charge, spin, mass etc.) of each elementary particle, but feel free to adopt more clever solutions!
- Reminder: all particles except quarks can be observed in isolation. Stable particles are: up, down, electron, all neutrinos, gluon, photon.

#### Solution
We need to choose how to deal with antiparticles. We want our class to exploit the fact that an antiparticle shares some properties with the corresponding particle, but we also want to provide to the user an intuitive interface where antiparticles have their own ids, starting with `anti-`. Internally, the class will store only the "original" particle identifier (without the `anti-` prefix) plus a boolean antiparticle attribute.

We still add to `from_id()` an argument to trigger the swap between particle and antiparticle, as it will be useful later (see `Hadron`).

In [ ]:
class ElementaryParticle(Particle):
    antiparticle_prefix = "anti"
    antiparticle_sep = '-'
    
    def __init__(self, *args, antiparticle, **kwargs):
        """ pass through all the arguments to the parent class constructor """
        self.antiparticle = antiparticle
        super().__init__(*args, **kwargs)

    @classmethod
    def parse_id(cls, id : str):
        # print(id)
        if id.startswith(cls.antiparticle_prefix):
            id = id.split(cls.antiparticle_sep)[1]
            antiparticle = True
        else:
            antiparticle = False
        return id, antiparticle

    def get_id(self):
        if self.antiparticle:
            return self.antiparticle_prefix + self.antiparticle_sep + self.id
        else:
            return self.id

    def get_charge(self):
        if self.antiparticle:
            return -1. * self.charge
        else:
            return self.charge

We now define the `Quark` class. We decide to tabulate some properties at the level of class attributes, while other properties are assigned in `from_id()`.

In [ ]:
class Quark(ElementaryParticle):

    quark_masses = {    
                        "up": 2.2e6,
                        "down": 4.7e6,
                        "charm": 1.28e9,
                        "strange": 96e6,
                        "top": 173.1e9,
                        "bottom": 4.18e9,
                    }

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_id(cls, id : str, anti_swap = False):
        # print(f'Creating quark of type {id}')

        id, antiparticle = cls.parse_id(id)

        if anti_swap:
            antiparticle = not antiparticle

        if not id in ['up', 'down', 'charm', 'strange', 'top', 'bottom']:
            return None

        mass = cls.quark_masses.get(id)
        spin = 0.5

        if id in ['up', 'charm', 'top']:
            charge = 2./3.
        elif id in ['down', 'strange', 'bottom']:
            charge = -1./3.
        else:
            charge = None

        stable = id in ['up', 'down']

        return Quark(id, mass, charge, spin, stable=stable, always_bound=True, antiparticle=antiparticle)

In [ ]:
q = Quark.from_id('up')
print(q)

q = Quark.from_id('anti-down')
print(q)

# Note that `__str__()`` is defined for `Particle` but when `get_id()` is called, the more specific method for `ElementaryParticle` is executed.

In [ ]:
class Lepton(ElementaryParticle):
    """
    Neutrino masses are just upper limits, for simplicity we will deal with them as they were actual masses.
    A more sophisticated approach would be to define a class for `ParticleMass` that can represent both measured masses and upper limits.
    """
    lepton_masses = { "e": 511e3, "mu": 105.6e6, "tau": 1776e9, "nu_e": 1.0, "nu_mu": 0.17, "nu_tau": 18.2e6 }

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def from_id(cls, id, antiparticle=False):
        id, antiparticle = cls.parse_id(id)
        
        if not id in ['e', 'mu', 'tau', 'nu_e', 'nu_mu', 'nu_tau']:
            return None

        spin = 0.5

        if id in ['e', 'mu', 'tau']:
            charge = -1
        else:
            charge = 0

        mass = cls.lepton_masses.get(id)

        return Lepton(id, mass, charge, spin, always_bound=False, antiparticle=antiparticle)

In [ ]:
l = Lepton.from_id('e')
print(l)
l = Lepton.from_id('anti-e')
print(l)

### 3. Hadrons
Define a class `Hadron` derived from `Particles` to describe particles made of quarks:
- `Meson`: pi+ (u anti-d), pi- (d anti-u), pi0 (u anti-u), K+ (u anti-s), K0 (d anti-s), K- (s anti-u);
- `Nucleon`: proton (u u d), neutron (u d d);

Notes:
- choose an unique identifier (a single letter or short string) for each type of particle in a category;
- make sure a `Hadron` object always contains a list of the objects corresponding to the particles composing it;
- remember that the electric charge of a hadron is the sum of the charges of the quarks composing it; 
- provide each class with a `from_id()` method that takes as an argument the unique identifier of a particle and returns an *instance* of such particle!
- provide `Hadron` with a class method that takes as an input a list of `Quark` objects and returns the corresponding `Meson` or `Nucleon` if the combination corresponds to a known particles (limit yourself to the particles in the above list).

In [ ]:
class Hadron(Particle):
    mesons =  ["pi+", "pi-", "pi0", "K+", "K-", "K0"]
    nucleons = ["p", "n"]

    def __init__(self, *args, quarks, antiparticle, **kwargs):
        self.quarks = quarks
        self.antiparticle = antiparticle
        charge = sum(p.get_charge() for p in quarks)
        super().__init__(*args, charge=charge, **kwargs)

    @classmethod
    def from_id(cls, id):
        if id in cls.mesons:
            return Meson.from_id(id)
        elif id in cls.nucleons:
            return Nucleon.from_id(id)
        else:
            return None

    @classmethod
    def from_quarks(cls, quarks : list):
        n_q = len(quarks)
        if n_q == 3:
            # the particle is a nucleon
            return Nucleon.from_quarks(quarks)
        elif n_q == 2:
            # the particle is a meson
            return Meson.from_quarks(quarks)
        else:
            print(f"Error: hadrons with a {n_q} quarks are not implemented.")
            return None
        
    @classmethod
    def match_composition(cls, quarks):
        composition = sorted(q.get_id() for q in quarks)
        for id in cls.composition_table:
            for comp in cls.composition_table[id]:
                if sorted(comp) == composition:
                    print(f"Match composition for hadron `{id}`.")
                    return id
                else:
                    # print(comp, composition)
                    # debug print(), disabled as soon as everything checkout
                    pass
        return None
    

We have not defined `Meson` or `Nucleon` yet, but let's thest if the attempt at creating a non-supported hadron is dealt with!

In [ ]:
quark_ids = ["up", "up", "down", "charm"]

quarks = [Quark.from_id(q) for q in quark_ids]

h = Hadron.from_quarks(quarks)

#### Solution: `Meson` and `Nucleon`
For `Meson` antiparticles, we will adopt a solution similar to what we have done for `ElementaryParticle`, with the difference that here we do not use an `anti-` prefix, but we use the suffixes `+` or `-` to identify the particle and the antiparticle (when they have a charge). Aside from that, the logic is the same: only the id exposed to the user is changed but the class uses a different internal representation.

To deal with the fact that some mesons allow more than one composition, we describe the possible compositions as a tuple of tuples. Then we build a dictionary with the reverse mapping from composition to id. Note that when a tuple contains a single tuple, a comma goes after the definition of tuple, i.e.: ((a,b),)

In [ ]:
class Meson(Hadron):
    mesons_masses =  { "pi+" : 139.6e6, "pi0" : 135.9e6, "K+" : 493.6e6, "K0" : 493.6e6 }

    composition_table = { "pi+" : (("up", "anti-down"),), "pi-" : (("down", "anti-up"),), "pi0" : (("up", "anti-up"), ("down", "anti-down")), "K+" : (("up", "anti-strange"),), "K-" : (("anti-up", "strange"),), "K0" : (("down", "anti-strange"),) }

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def parse_id(cls, id):
        antiparticle = False
        if id.endswith("-"):
            # replace last character (-) with + and return a new string
            id = id[:-1] + "+" 
            antiparticle = True
        return id, antiparticle
    
    def get_id(self):
        id = self.id
        if self.antiparticle:
            if self.id.endswith("+"):
                id = id[:-1] + "-"
            else:
                """
                This situation, with the id ending in "-" and antiparticle set to True should not occur.
                However, the unexpected may happen and better report to the user rather than silently ignore.
                """
                print("Unexpected combination of internal id and antiparticle status.")
                print("Probably a bug you may want to report!")
                return None
        return id

    @classmethod
    def from_id(cls, id : str, quarks=None):
        if not id in Hadron.mesons:
            return None
        
        id, antiparticle = cls.parse_id(id)

        # print(id, antiparticle)

        if quarks is None:
            comps = cls.composition_table.get(id)
            quarks = [Quark.from_id(qid, anti_swap=antiparticle) for comp in comps for qid in comp]
            # note the nested loop syntax in list comprehension

        mass = cls.mesons_masses.get(id)
        spin = 0
        stability = False
        always_bound = True

        return Meson(id, quarks=quarks, antiparticle=antiparticle, mass=mass, spin=spin, stable=stability, always_bound=always_bound)
    
    @classmethod
    def from_quarks(cls, quarks):
        id = cls.match_composition(quarks)
        return(cls.from_id(id, quarks))

In [ ]:
quark_combos = (["up", "anti-down"],["down", "anti-up"])

for combo in quark_combos:
    quarks = [Quark.from_id(q) for q in combo]
    h = Hadron.from_quarks(quarks)
    print(h)


In [ ]:
for id in Hadron.mesons:
    h = Hadron.from_id(id)
    print(id, "=>", h)

For `Nucleon` we use the same convention of `ElementaryParticle`. For simplicity, we will just copy the `parse_id` and `get_id` methods, however ideally they could be moved to `Particle` so they can actually be reused!

In [ ]:
class Nucleon(Hadron):
    antiparticle_prefix = "anti"
    antiparticle_sep = "-"

    composition_table = { "p" : (("up", "up", "down"),), "n" : (("up","down","down"),) } 

    nucleons_masses = { "p" : 1e9, "n": 1e9 }
    nucleons_stability = { "p" : True, "n" : False }

    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)

    @classmethod
    def parse_id(cls, id : str):
        # print(id)
        if id.startswith(cls.antiparticle_prefix):
            id = id.split(cls.antiparticle_sep)[1]
            antiparticle = True
        else:
            antiparticle = False
        return id, antiparticle

    def get_id(self):
        if self.antiparticle:
            return self.antiparticle_prefix + self.antiparticle_sep + self.id
        else:
            return self.id
        
    @classmethod
    def from_id(cls, id, quarks = None):        
        id, antiparticle = cls.parse_id(id)

        if not id in Hadron.nucleons:
            return None

        mass = cls.nucleons_masses.get(id)
        stability = cls.nucleons_stability.get(id)
        
        if quarks is None:
            comps = cls.composition_table.get(id)
            quarks = [Quark.from_id(qid, anti_swap=antiparticle) for comp in comps for qid in comp]

        return Nucleon(id, quarks=quarks, mass=mass, spin=0.5, stable=stability, always_bound=False, antiparticle=antiparticle)

    @classmethod
    def from_quarks(cls, quarks):
        id = cls.match_composition(quarks)
        return(cls.from_id(id, quarks))

In [ ]:
ids = ['p', 'anti-p', 'n', 'anti-n']

for id in ids:
    n = Nucleon.from_id(id)
    print(id, "=>", n)

qids = ["up", "up", "down"]

quarks = [Quark.from_id(qid) for qid in qids]

p = Nucleon.from_quarks(quarks)

print(p)


### 4/Bonus: Atomic nuclei
*This exercise is optional and will allow you to gain bonus points!*

Provide your own creative design of a class `Nucleus` to represent atomic nuclei!